# Обратное распространение ошибки (Backpropagation)

В этой тетрадке мы детально разберем:
- Математику обратного распространения
- Вычислительный граф и цепное правило
- Пошаговую реализацию с нуля
- Численную проверку градиентов
- Визуализацию процесса обучения
- Практические примеры

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch
from sklearn.datasets import make_moons, make_circles
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# Настройка графиков
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## 1. Введение: Зачем нужно обратное распространение?

### Задача обучения нейронной сети:

Дано:
- Обучающие данные: $(x^{(i)}, y^{(i)})$, где $i = 1, ..., m$
- Нейронная сеть с параметрами (весами и смещениями)
- Функция потерь $L(\hat{y}, y)$

Найти:
- Оптимальные значения параметров, минимизирующие функцию потерь

### Метод градиентного спуска:

$$w := w - \alpha \frac{\partial L}{\partial w}$$

где $\alpha$ — learning rate (скорость обучения).

**Проблема:** Как вычислить $\frac{\partial L}{\partial w}$ для всех весов в сети?

**Решение:** Алгоритм обратного распространения ошибки (backpropagation)!

## 2. Математические основы: Цепное правило

### Цепное правило для композиции функций:

Если $y = f(u)$ и $u = g(x)$, то:
$$\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx}$$

### Для многомерного случая:

Если $z = f(x, y)$, $x = g(t)$, $y = h(t)$, то:
$$\frac{dz}{dt} = \frac{\partial z}{\partial x} \frac{dx}{dt} + \frac{\partial z}{\partial y} \frac{dy}{dt}$$

### Пример:

In [ ]:
def chain_rule_example():
    """
    Пример: f(x) = (3x + 2)^2
    
    Представим как композицию:
    u = 3x + 2
    f = u^2
    
    Тогда: df/dx = df/du * du/dx = 2u * 3 = 6u = 6(3x + 2)
    """
    
    x = 2.0
    
    # Прямое распространение
    u = 3 * x + 2  # u = 8
    f = u ** 2      # f = 64
    
    print("Прямое распространение:")
    print(f"x = {x}")
    print(f"u = 3x + 2 = {u}")
    print(f"f = u^2 = {f}")
    
    # Обратное распространение
    df_du = 2 * u   # df/du = 2u = 16
    du_dx = 3       # du/dx = 3
    df_dx = df_du * du_dx  # df/dx = 48
    
    print("\nОбратное распространение (цепное правило):")
    print(f"df/du = 2u = {df_du}")
    print(f"du/dx = 3 = {du_dx}")
    print(f"df/dx = df/du * du/dx = {df_dx}")
    
    # Проверка численным методом
    epsilon = 1e-7
    f_plus = (3 * (x + epsilon) + 2) ** 2
    f_minus = (3 * (x - epsilon) + 2) ** 2
    df_dx_numerical = (f_plus - f_minus) / (2 * epsilon)
    
    print("\nЧисленная проверка:")
    print(f"df/dx (численно) = {df_dx_numerical:.6f}")
    print(f"df/dx (аналитически) = {df_dx}")
    print(f"Разница: {abs(df_dx - df_dx_numerical):.10f}")

chain_rule_example()

## 3. Вычислительный граф

Вычислительный граф — это направленный граф, где:
- **Узлы** — операции или переменные
- **Рёбра** — поток данных

### Пример вычислительного графа:

In [ ]:
def visualize_computational_graph():
    """Визуализация вычислительного графа для f = (x*w + b)^2"""
    
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Координаты узлов
    nodes = {
        'x': (1, 3),
        'w': (1, 1),
        'b': (3, 1),
        'mul': (2, 2),
        'add': (4, 2),
        'square': (6, 2),
        'L': (8, 2)
    }
    
    # Прямое распространение (синие стрелки)
    forward_edges = [
        ('x', 'mul', 'x=2'),
        ('w', 'mul', 'w=3'),
        ('mul', 'add', 'x*w=6'),
        ('b', 'add', 'b=1'),
        ('add', 'square', 'x*w+b=7'),
        ('square', 'L', '(x*w+b)²=49')
    ]
    
    # Обратное распространение (красные стрелки)
    backward_edges = [
        ('L', 'square', 'dL/d(•)=1'),
        ('square', 'add', 'dL/d(•)=14'),
        ('add', 'mul', 'dL/d(•)=14'),
        ('add', 'b', 'dL/db=14'),
        ('mul', 'w', 'dL/dw=28'),
        ('mul', 'x', 'dL/dx=42')
    ]
    
    # Рисуем узлы
    for name, (x, y) in nodes.items():
        if name in ['x', 'w', 'b']:
            color = 'lightblue'
            label = f'{name}'
        elif name == 'L':
            color = 'lightcoral'
            label = 'Loss'
        else:
            color = 'lightgreen'
            if name == 'mul':
                label = '×'
            elif name == 'add':
                label = '+'
            else:
                label = '²'
        
        circle = plt.Circle((x, y), 0.3, color=color, ec='black', linewidth=2, zorder=3)
        ax.add_patch(circle)
        ax.text(x, y, label, ha='center', va='center', fontsize=14, fontweight='bold', zorder=4)
    
    # Рисуем прямые рёбра
    for src, dst, label in forward_edges:
        x1, y1 = nodes[src]
        x2, y2 = nodes[dst]
        
        # Стрелка
        dx = x2 - x1
        dy = y2 - y1
        length = np.sqrt(dx**2 + dy**2)
        dx_norm = dx / length * 0.3
        dy_norm = dy / length * 0.3
        
        ax.arrow(x1 + dx_norm, y1 + dy_norm, 
                dx - 2*dx_norm, dy - 2*dy_norm,
                head_width=0.15, head_length=0.1, 
                fc='blue', ec='blue', linewidth=2, zorder=1)
        
        # Метка
        ax.text((x1 + x2) / 2, (y1 + y2) / 2 + 0.3, label, 
               ha='center', fontsize=9, color='blue', zorder=2)
    
    # Рисуем обратные рёбра (пунктиром)
    for src, dst, label in backward_edges:
        x1, y1 = nodes[src]
        x2, y2 = nodes[dst]
        
        # Стрелка (смещена вниз)
        offset = -0.2
        dx = x2 - x1
        dy = y2 - y1
        length = np.sqrt(dx**2 + dy**2)
        dx_norm = dx / length * 0.3
        dy_norm = dy / length * 0.3
        
        ax.arrow(x1 + dx_norm, y1 + dy_norm + offset, 
                dx - 2*dx_norm, dy - 2*dy_norm,
                head_width=0.15, head_length=0.1,
                fc='red', ec='red', linewidth=2, 
                linestyle='--', alpha=0.7, zorder=1)
        
        # Метка
        ax.text((x1 + x2) / 2, (y1 + y2) / 2 - 0.4, label,
               ha='center', fontsize=9, color='red', zorder=2)
    
    # Легенда
    ax.text(1, 4.2, 'Вычислительный граф', fontsize=16, fontweight='bold')
    ax.text(1, 3.8, '→ Прямое распространение (forward)', color='blue', fontsize=11)
    ax.text(1, 3.5, '⇢ Обратное распространение (backward)', color='red', fontsize=11)
    
    ax.set_xlim(0, 9)
    ax.set_ylim(0, 4.5)
    ax.axis('off')
    plt.tight_layout()
    plt.show()

visualize_computational_graph()

## 4. Обратное распространение в простой сети

Рассмотрим простую сеть с одним скрытым слоем:

```
Input (x) → Hidden (h) → Output (ŷ) → Loss (L)
```

### Архитектура:

**Слой 1 (вход → скрытый):**
$$z^{[1]} = W^{[1]} x + b^{[1]}$$
$$a^{[1]} = \sigma(z^{[1]})$$

**Слой 2 (скрытый → выход):**
$$z^{[2]} = W^{[2]} a^{[1]} + b^{[2]}$$
$$a^{[2]} = \sigma(z^{[2]}) = \hat{y}$$

**Функция потерь (MSE):**
$$L = \frac{1}{2}(\hat{y} - y)^2$$

### Прямое распространение (Forward Pass):

1. Вычислить $z^{[1]}, a^{[1]}$
2. Вычислить $z^{[2]}, a^{[2]}$
3. Вычислить $L$

### Обратное распространение (Backward Pass):

**Выходной слой:**
$$\frac{\partial L}{\partial a^{[2]}} = \hat{y} - y$$

$$\frac{\partial L}{\partial z^{[2]}} = \frac{\partial L}{\partial a^{[2]}} \cdot \sigma'(z^{[2]})$$

$$\frac{\partial L}{\partial W^{[2]}} = \frac{\partial L}{\partial z^{[2]}} \cdot (a^{[1]})^T$$

$$\frac{\partial L}{\partial b^{[2]}} = \frac{\partial L}{\partial z^{[2]}}$$

**Скрытый слой:**
$$\frac{\partial L}{\partial a^{[1]}} = (W^{[2]})^T \cdot \frac{\partial L}{\partial z^{[2]}}$$

$$\frac{\partial L}{\partial z^{[1]}} = \frac{\partial L}{\partial a^{[1]}} \cdot \sigma'(z^{[1]})$$

$$\frac{\partial L}{\partial W^{[1]}} = \frac{\partial L}{\partial z^{[1]}} \cdot x^T$$

$$\frac{\partial L}{\partial b^{[1]}} = \frac{\partial L}{\partial z^{[1]}}$$

## 5. Пошаговая реализация с детальными выводами

In [ ]:
class NeuralNetworkVerbose:
    """Нейронная сеть с подробным выводом вычислений"""
    
    def __init__(self, input_size, hidden_size, output_size):
        # Инициализация весов
        self.W1 = np.random.randn(input_size, hidden_size) * 0.5
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.5
        self.b2 = np.zeros((1, output_size))
        
        print("Инициализация сети:")
        print(f"W1 shape: {self.W1.shape}, b1 shape: {self.b1.shape}")
        print(f"W2 shape: {self.W2.shape}, b2 shape: {self.b2.shape}")
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, z):
        s = self.sigmoid(z)
        return s * (1 - s)
    
    def forward_verbose(self, X, y):
        """Прямое распространение с подробным выводом"""
        
        print("\n" + "="*60)
        print("ПРЯМОЕ РАСПРОСТРАНЕНИЕ (FORWARD PASS)")
        print("="*60)
        
        # Слой 1
        print("\n--- Слой 1: Вход → Скрытый ---")
        print(f"Вход X shape: {X.shape}")
        
        self.z1 = np.dot(X, self.W1) + self.b1
        print(f"z1 = X @ W1 + b1")
        print(f"z1 shape: {self.z1.shape}")
        print(f"z1[0, :3] = {self.z1[0, :3]}")
        
        self.a1 = self.sigmoid(self.z1)
        print(f"\na1 = sigmoid(z1)")
        print(f"a1[0, :3] = {self.a1[0, :3]}")
        
        # Слой 2
        print("\n--- Слой 2: Скрытый → Выход ---")
        
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        print(f"z2 = a1 @ W2 + b2")
        print(f"z2 shape: {self.z2.shape}")
        print(f"z2[0] = {self.z2[0]}")
        
        self.a2 = self.sigmoid(self.z2)
        print(f"\na2 = sigmoid(z2) = ŷ (предсказание)")
        print(f"a2[0] = {self.a2[0]}")
        
        # Loss
        print("\n--- Вычисление функции потерь ---")
        self.loss = np.mean((self.a2 - y) ** 2)
        print(f"Loss = MSE = mean((ŷ - y)²)")
        print(f"Loss = {self.loss:.6f}")
        
        return self.a2
    
    def backward_verbose(self, X, y):
        """Обратное распространение с подробным выводом"""
        
        print("\n" + "="*60)
        print("ОБРАТНОЕ РАСПРОСТРАНЕНИЕ (BACKWARD PASS)")
        print("="*60)
        
        m = X.shape[0]
        
        # Градиенты выходного слоя
        print("\n--- Слой 2: Градиенты выходного слоя ---")
        
        # dL/da2
        dL_da2 = 2 * (self.a2 - y) / m
        print(f"dL/da2 = 2(ŷ - y) / m")
        print(f"dL/da2[0] = {dL_da2[0]}")
        
        # dL/dz2 = dL/da2 * da2/dz2
        da2_dz2 = self.sigmoid_derivative(self.z2)
        dL_dz2 = dL_da2 * da2_dz2
        print(f"\ndL/dz2 = dL/da2 ⊙ sigmoid'(z2)  (⊙ = поэлементное умножение)")
        print(f"dL/dz2[0] = {dL_dz2[0]}")
        
        # dL/dW2 = a1^T @ dL/dz2
        dL_dW2 = np.dot(self.a1.T, dL_dz2)
        print(f"\ndL/dW2 = a1.T @ dL/dz2")
        print(f"dL/dW2 shape: {dL_dW2.shape}")
        print(f"dL/dW2[:3, 0] = {dL_dW2[:3, 0]}")
        
        # dL/db2
        dL_db2 = np.sum(dL_dz2, axis=0, keepdims=True)
        print(f"\ndL/db2 = sum(dL/dz2, axis=0)")
        print(f"dL/db2 = {dL_db2}")
        
        # Градиенты скрытого слоя
        print("\n--- Слой 1: Градиенты скрытого слоя ---")
        
        # dL/da1 = dL/dz2 @ W2^T
        dL_da1 = np.dot(dL_dz2, self.W2.T)
        print(f"dL/da1 = dL/dz2 @ W2.T  (обратное распространение через W2)")
        print(f"dL/da1 shape: {dL_da1.shape}")
        print(f"dL/da1[0, :3] = {dL_da1[0, :3]}")
        
        # dL/dz1 = dL/da1 * da1/dz1
        da1_dz1 = self.sigmoid_derivative(self.z1)
        dL_dz1 = dL_da1 * da1_dz1
        print(f"\ndL/dz1 = dL/da1 ⊙ sigmoid'(z1)")
        print(f"dL/dz1[0, :3] = {dL_dz1[0, :3]}")
        
        # dL/dW1 = X^T @ dL/dz1
        dL_dW1 = np.dot(X.T, dL_dz1)
        print(f"\ndL/dW1 = X.T @ dL/dz1")
        print(f"dL/dW1 shape: {dL_dW1.shape}")
        print(f"dL/dW1[:3, :3] = \n{dL_dW1[:3, :3]}")
        
        # dL/db1
        dL_db1 = np.sum(dL_dz1, axis=0, keepdims=True)
        print(f"\ndL/db1 = sum(dL/dz1, axis=0)")
        print(f"dL/db1[:3] = {dL_db1[0, :3]}")
        
        return dL_dW1, dL_db1, dL_dW2, dL_db2
    
    def update_weights(self, dW1, db1, dW2, db2, learning_rate):
        """Обновление весов методом градиентного спуска"""
        
        print("\n" + "="*60)
        print("ОБНОВЛЕНИЕ ВЕСОВ (GRADIENT DESCENT)")
        print("="*60)
        
        print(f"\nLearning rate α = {learning_rate}")
        
        print("\nПеред обновлением:")
        print(f"W1[0, 0] = {self.W1[0, 0]:.6f}")
        print(f"W2[0, 0] = {self.W2[0, 0]:.6f}")
        
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        
        print("\nПосле обновления:")
        print(f"W1[0, 0] = {self.W1[0, 0]:.6f}  (изменение: {-learning_rate * dW1[0, 0]:.6f})")
        print(f"W2[0, 0] = {self.W2[0, 0]:.6f}  (изменение: {-learning_rate * dW2[0, 0]:.6f})")


# Демонстрация на простом примере
print("\n" + "#"*60)
print("# ДЕМОНСТРАЦИЯ ОБРАТНОГО РАСПРОСТРАНЕНИЯ")
print("#"*60)

# Простые данные
X = np.array([[0.5, 0.3]])
y = np.array([[1.0]])

print(f"\nВходные данные: X = {X}")
print(f"Целевое значение: y = {y}")

# Создание сети
nn = NeuralNetworkVerbose(input_size=2, hidden_size=3, output_size=1)

# Прямое распространение
predictions = nn.forward_verbose(X, y)

# Обратное распространение
dW1, db1, dW2, db2 = nn.backward_verbose(X, y)

# Обновление весов
nn.update_weights(dW1, db1, dW2, db2, learning_rate=0.1)

## 6. Численная проверка градиентов (Gradient Checking)

Для проверки правильности реализации обратного распространения используем численное дифференцирование:

$$\frac{\partial L}{\partial w} \approx \frac{L(w + \epsilon) - L(w - \epsilon)}{2\epsilon}$$

где $\epsilon$ — малое число (например, $10^{-7}$).

In [ ]:
def gradient_check(nn, X, y, epsilon=1e-7):
    """
    Численная проверка градиентов
    
    Сравнивает аналитические градиенты (из backprop) с численными
    """
    
    print("\n" + "="*60)
    print("ЧИСЛЕННАЯ ПРОВЕРКА ГРАДИЕНТОВ (GRADIENT CHECKING)")
    print("="*60)
    
    # Получаем аналитические градиенты
    nn.forward_verbose = lambda X, y: nn.sigmoid(np.dot(nn.sigmoid(np.dot(X, nn.W1) + nn.b1), nn.W2) + nn.b2)
    
    # Вычисляем loss
    def compute_loss(nn, X, y):
        z1 = np.dot(X, nn.W1) + nn.b1
        a1 = nn.sigmoid(z1)
        z2 = np.dot(a1, nn.W2) + nn.b2
        a2 = nn.sigmoid(z2)
        return np.mean((a2 - y) ** 2)
    
    # Аналитические градиенты через backprop
    z1 = np.dot(X, nn.W1) + nn.b1
    a1 = nn.sigmoid(z1)
    z2 = np.dot(a1, nn.W2) + nn.b2
    a2 = nn.sigmoid(z2)
    
    m = X.shape[0]
    dL_da2 = 2 * (a2 - y) / m
    dL_dz2 = dL_da2 * nn.sigmoid_derivative(z2)
    dL_dW2_analytical = np.dot(a1.T, dL_dz2)
    
    dL_da1 = np.dot(dL_dz2, nn.W2.T)
    dL_dz1 = dL_da1 * nn.sigmoid_derivative(z1)
    dL_dW1_analytical = np.dot(X.T, dL_dz1)
    
    # Численные градиенты для W1[0, 0]
    print("\nПроверка градиента для W1[0, 0]:")
    
    original_value = nn.W1[0, 0]
    
    # L(w + epsilon)
    nn.W1[0, 0] = original_value + epsilon
    loss_plus = compute_loss(nn, X, y)
    
    # L(w - epsilon)
    nn.W1[0, 0] = original_value - epsilon
    loss_minus = compute_loss(nn, X, y)
    
    # Восстанавливаем
    nn.W1[0, 0] = original_value
    
    # Численный градиент
    dL_dW1_numerical = (loss_plus - loss_minus) / (2 * epsilon)
    dL_dW1_analytical_value = dL_dW1_analytical[0, 0]
    
    print(f"Аналитический градиент: {dL_dW1_analytical_value:.10f}")
    print(f"Численный градиент:     {dL_dW1_numerical:.10f}")
    print(f"Разница:                {abs(dL_dW1_analytical_value - dL_dW1_numerical):.2e}")
    
    # Проверка градиента для W2[0, 0]
    print("\nПроверка градиента для W2[0, 0]:")
    
    original_value = nn.W2[0, 0]
    
    nn.W2[0, 0] = original_value + epsilon
    loss_plus = compute_loss(nn, X, y)
    
    nn.W2[0, 0] = original_value - epsilon
    loss_minus = compute_loss(nn, X, y)
    
    nn.W2[0, 0] = original_value
    
    dL_dW2_numerical = (loss_plus - loss_minus) / (2 * epsilon)
    dL_dW2_analytical_value = dL_dW2_analytical[0, 0]
    
    print(f"Аналитический градиент: {dL_dW2_analytical_value:.10f}")
    print(f"Численный градиент:     {dL_dW2_numerical:.10f}")
    print(f"Разница:                {abs(dL_dW2_analytical_value - dL_dW2_numerical):.2e}")
    
    print("\n✓ Если разница < 1e-7, то градиенты вычислены правильно!")

# Проверка
gradient_check(nn, X, y)

## 7. Полная реализация с визуализацией обучения

In [ ]:
class NeuralNetworkBackprop:
    """Полная реализация нейронной сети с backpropagation"""
    
    def __init__(self, layer_sizes):
        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes) - 1
        
        # Инициализация весов (He инициализация)
        self.weights = []
        self.biases = []
        
        for i in range(self.num_layers):
            w = np.random.randn(layer_sizes[i], layer_sizes[i+1]) * np.sqrt(2.0 / layer_sizes[i])
            b = np.zeros((1, layer_sizes[i+1]))
            self.weights.append(w)
            self.biases.append(b)
        
        # История обучения
        self.losses = []
        self.gradient_norms = []
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, z):
        s = self.sigmoid(z)
        return s * (1 - s)
    
    def relu(self, z):
        return np.maximum(0, z)
    
    def relu_derivative(self, z):
        return (z > 0).astype(float)
    
    def forward(self, X):
        """Прямое распространение"""
        self.activations = [X]
        self.zs = []
        
        A = X
        for i in range(self.num_layers):
            Z = np.dot(A, self.weights[i]) + self.biases[i]
            self.zs.append(Z)
            
            # Используем ReLU для скрытых слоев, sigmoid для выходного
            if i == self.num_layers - 1:
                A = self.sigmoid(Z)
            else:
                A = self.relu(Z)
            
            self.activations.append(A)
        
        return A
    
    def backward(self, X, y):
        """Обратное распространение"""
        m = X.shape[0]
        
        # Инициализация градиентов
        dW = [np.zeros_like(w) for w in self.weights]
        db = [np.zeros_like(b) for b in self.biases]
        
        # Градиент выходного слоя (для MSE)
        dA = 2 * (self.activations[-1] - y) / m
        
        # Обратное распространение через слои
        for i in reversed(range(self.num_layers)):
            # Градиент по z
            if i == self.num_layers - 1:
                dZ = dA * self.sigmoid_derivative(self.zs[i])
            else:
                dZ = dA * self.relu_derivative(self.zs[i])
            
            # Градиенты по весам и смещениям
            dW[i] = np.dot(self.activations[i].T, dZ)
            db[i] = np.sum(dZ, axis=0, keepdims=True)
            
            # Градиент для предыдущего слоя
            if i > 0:
                dA = np.dot(dZ, self.weights[i].T)
        
        # Сохраняем норму градиентов для визуализации
        grad_norm = sum(np.sum(g**2) for g in dW)
        self.gradient_norms.append(grad_norm)
        
        return dW, db
    
    def update_weights(self, dW, db, learning_rate):
        """Обновление весов"""
        for i in range(self.num_layers):
            self.weights[i] -= learning_rate * dW[i]
            self.biases[i] -= learning_rate * db[i]
    
    def compute_loss(self, y_pred, y_true):
        """Вычисление MSE loss"""
        return np.mean((y_pred - y_true) ** 2)
    
    def train(self, X, y, epochs, learning_rate, verbose=True):
        """Обучение сети"""
        for epoch in range(epochs):
            # Forward pass
            y_pred = self.forward(X)
            
            # Compute loss
            loss = self.compute_loss(y_pred, y)
            self.losses.append(loss)
            
            # Backward pass
            dW, db = self.backward(X, y)
            
            # Update weights
            self.update_weights(dW, db, learning_rate)
            
            if verbose and epoch % 100 == 0:
                accuracy = np.mean((y_pred > 0.5).astype(int) == y)
                print(f'Epoch {epoch}: Loss = {loss:.6f}, Accuracy = {accuracy:.4f}')
    
    def predict(self, X):
        """Предсказание"""
        y_pred = self.forward(X)
        return (y_pred > 0.5).astype(int)

## 8. Обучение на реальных данных

In [ ]:
# Генерация данных (полумесяцы)
X, y = make_moons(n_samples=300, noise=0.2, random_state=42)
y = y.reshape(-1, 1)

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")

# Визуализация данных
plt.figure(figsize=(8, 6))
plt.scatter(X_train[y_train.flatten() == 0, 0], X_train[y_train.flatten() == 0, 1],
           c='red', marker='o', s=50, alpha=0.7, edgecolors='k', label='Класс 0')
plt.scatter(X_train[y_train.flatten() == 1, 0], X_train[y_train.flatten() == 1, 1],
           c='blue', marker='s', s=50, alpha=0.7, edgecolors='k', label='Класс 1')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.title('Обучающие данные (полумесяцы)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Создание и обучение сети
print("\nОбучение нейронной сети...\n")

nn = NeuralNetworkBackprop(layer_sizes=[2, 16, 8, 1])
nn.train(X_train, y_train, epochs=1000, learning_rate=0.1, verbose=True)

# Оценка на тестовой выборке
y_pred_test = nn.predict(X_test)
test_accuracy = np.mean(y_pred_test == y_test)
print(f"\nТочность на тестовой выборке: {test_accuracy:.4f}")

In [ ]:
# Визуализация процесса обучения
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# График функции потерь
axes[0].plot(nn.losses, linewidth=2)
axes[0].set_xlabel('Эпоха')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Кривая обучения: функция потерь')
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')

# График нормы градиентов
axes[1].plot(nn.gradient_norms, linewidth=2, color='orange')
axes[1].set_xlabel('Эпоха')
axes[1].set_ylabel('Норма градиента')
axes[1].set_title('Эволюция нормы градиентов')
axes[1].grid(True, alpha=0.3)
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

In [ ]:
# Визуализация границы решений
def plot_decision_boundary(model, X, y, title):
    """Визуализация границы решений"""
    
    h = 0.02  # Шаг сетки
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Предсказания для всех точек сетки
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Визуализация
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, alpha=0.4, cmap='RdBu')
    plt.contour(xx, yy, Z, colors='black', linewidths=0.5, levels=[0.5])
    
    # Точки данных
    plt.scatter(X[y.flatten() == 0, 0], X[y.flatten() == 0, 1],
               c='red', marker='o', s=50, alpha=0.7, edgecolors='k', label='Класс 0')
    plt.scatter(X[y.flatten() == 1, 0], X[y.flatten() == 1, 1],
               c='blue', marker='s', s=50, alpha=0.7, edgecolors='k', label='Класс 1')
    
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xlabel('Признак 1')
    plt.ylabel('Признак 2')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

plot_decision_boundary(nn, X_train, y_train, 
                      'Граница решений после обучения')

## 9. Визуализация активаций по слоям

In [ ]:
def visualize_activations(model, X, sample_idx=0):
    """Визуализация активаций для одного примера"""
    
    # Прямое распространение
    model.forward(X)
    
    # Визуализация
    fig, axes = plt.subplots(1, len(model.activations), figsize=(15, 3))
    
    for i, (ax, activation) in enumerate(zip(axes, model.activations)):
        values = activation[sample_idx].reshape(-1, 1)
        
        im = ax.imshow(values.T, cmap='viridis', aspect='auto')
        ax.set_yticks([])
        ax.set_xlabel('Нейрон')
        
        if i == 0:
            ax.set_title(f'Вход\n(размер: {values.shape[0]})')
        elif i == len(model.activations) - 1:
            ax.set_title(f'Выход\n(размер: {values.shape[0]})')
        else:
            ax.set_title(f'Слой {i}\n(размер: {values.shape[0]})')
        
        plt.colorbar(im, ax=ax, fraction=0.046)
    
    plt.suptitle(f'Активации для примера {sample_idx}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Визуализация для нескольких примеров
for idx in [0, 5, 10]:
    visualize_activations(nn, X_train, sample_idx=idx)

## 10. Сравнение с разными функциями активации

In [ ]:
# Генерация более сложных данных (концентрические окружности)
X_circles, y_circles = make_circles(n_samples=300, noise=0.1, factor=0.5, random_state=42)
y_circles = y_circles.reshape(-1, 1)

# Визуализация
plt.figure(figsize=(8, 6))
plt.scatter(X_circles[y_circles.flatten() == 0, 0], X_circles[y_circles.flatten() == 0, 1],
           c='red', marker='o', s=50, alpha=0.7, edgecolors='k', label='Класс 0')
plt.scatter(X_circles[y_circles.flatten() == 1, 0], X_circles[y_circles.flatten() == 1, 1],
           c='blue', marker='s', s=50, alpha=0.7, edgecolors='k', label='Класс 1')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.title('Данные: концентрические окружности')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Обучение сети на новых данных
print("Обучение на концентрических окружностях...\n")

nn_circles = NeuralNetworkBackprop(layer_sizes=[2, 32, 16, 8, 1])
nn_circles.train(X_circles, y_circles, epochs=2000, learning_rate=0.1, verbose=True)

# Оценка
y_pred_circles = nn_circles.predict(X_circles)
accuracy = np.mean(y_pred_circles == y_circles)
print(f"\nТочность: {accuracy:.4f}")

# Визуализация границы решений
plot_decision_boundary(nn_circles, X_circles, y_circles,
                      'Граница решений для концентрических окружностей')

## 11. Ключевые выводы

### Алгоритм обратного распространения:

**Шаг 1: Прямое распространение (Forward Pass)**
- Вычисляем активации всех слоёв
- Сохраняем промежуточные значения $z^{[l]}$ и $a^{[l]}$
- Вычисляем функцию потерь $L$

**Шаг 2: Обратное распространение (Backward Pass)**
- Начинаем с выходного слоя: вычисляем $\frac{\partial L}{\partial z^{[L]}}$
- Двигаемся назад через слои, используя цепное правило
- Для каждого слоя вычисляем: $\frac{\partial L}{\partial W^{[l]}}$, $\frac{\partial L}{\partial b^{[l]}}$

**Шаг 3: Обновление весов (Gradient Descent)**
$$W^{[l]} := W^{[l]} - \alpha \frac{\partial L}{\partial W^{[l]}}$$
$$b^{[l]} := b^{[l]} - \alpha \frac{\partial L}{\partial b^{[l]}}$$

### Преимущества backpropagation:

| Свойство | Описание |
|----------|----------|
| **Эффективность** | Вычисляет все градиенты за один проход (O(n)) |
| **Точность** | Аналитические градиенты точнее численных |
| **Универсальность** | Работает для любой дифференцируемой архитектуры |
| **Модульность** | Легко добавлять новые слои и операции |

### Важные моменты:

1. **Цепное правило** — основа backpropagation
2. **Вычислительный граф** — удобное представление вычислений
3. **Кэширование** — сохраняем промежуточные значения из forward pass
4. **Gradient checking** — проверяем правильность реализации
5. **Векторизация** — используем матричные операции для эффективности

### Типичные ошибки:

1. ❌ Неправильные размерности матриц
2. ❌ Забыли сохранить промежуточные значения
3. ❌ Неправильный порядок умножения матриц
4. ❌ Забыли применить производную функции активации
5. ❌ Неправильное усреднение по батчу

### Оптимизации:

- **Mini-batch SGD** — компромисс между скоростью и стабильностью
- **Momentum** — сглаживание градиентов
- **Adam** — адаптивная скорость обучения
- **Learning rate scheduling** — уменьшение LR со временем

### Практические советы:

1. Всегда проверяйте градиенты численно (gradient checking)
2. Визуализируйте кривую обучения
3. Мониторьте норму градиентов
4. Используйте правильную инициализацию весов
5. Нормализуйте входные данные
6. Начинайте с малой сети и увеличивайте при необходимости

## 12. Математическая сводка

### Прямое распространение:

Для слоя $l$:
$$z^{[l]} = W^{[l]} a^{[l-1]} + b^{[l]}$$
$$a^{[l]} = g^{[l]}(z^{[l]})$$

где $g^{[l]}$ — функция активации слоя $l$.

### Обратное распространение:

**Выходной слой** (для MSE loss):
$$\frac{\partial L}{\partial z^{[L]}} = (a^{[L]} - y) \odot g'(z^{[L]})$$

**Скрытые слои** (для $l = L-1, ..., 1$):
$$\frac{\partial L}{\partial z^{[l]}} = \left((W^{[l+1]})^T \frac{\partial L}{\partial z^{[l+1]}}\right) \odot g'(z^{[l]})$$

**Градиенты параметров:**
$$\frac{\partial L}{\partial W^{[l]}} = \frac{1}{m} \frac{\partial L}{\partial z^{[l]}} (a^{[l-1]})^T$$
$$\frac{\partial L}{\partial b^{[l]}} = \frac{1}{m} \sum_{i=1}^{m} \frac{\partial L}{\partial z^{[l]}_i}$$

где:
- $\odot$ — поэлементное произведение (Hadamard product)
- $m$ — размер батча
- $g'$ — производная функции активации

### Производные функций активации:

**Sigmoid:**
$$\sigma'(z) = \sigma(z)(1 - \sigma(z))$$

**Tanh:**
$$\tanh'(z) = 1 - \tanh^2(z)$$

**ReLU:**
$$\text{ReLU}'(z) = \begin{cases} 1, & z > 0 \\ 0, & z \leq 0 \end{cases}$$

### Функции потерь:

**MSE (регрессия):**
$$L = \frac{1}{2m} \sum_{i=1}^{m} (\hat{y}_i - y_i)^2$$
$$\frac{\partial L}{\partial \hat{y}} = \frac{1}{m}(\hat{y} - y)$$

**Binary Cross-Entropy (бинарная классификация):**
$$L = -\frac{1}{m} \sum_{i=1}^{m} [y_i \log(\hat{y}_i) + (1-y_i)\log(1-\hat{y}_i)]$$
$$\frac{\partial L}{\partial \hat{y}} = \frac{1}{m}\left(\frac{\hat{y} - y}{\hat{y}(1-\hat{y})}\right)$$

## 13. Задания для самостоятельной работы

1. **Реализация:**
   - Добавьте поддержку различных функций потерь (BCE, categorical cross-entropy)
   - Реализуйте mini-batch gradient descent
   - Добавьте momentum и Adam optimizer

2. **Градиенты:**
   - Реализуйте gradient checking для всех весов сети
   - Визуализируйте распределение градиентов по слоям
   - Исследуйте проблему затухающих/взрывающихся градиентов

3. **Архитектура:**
   - Добавьте L2-регуляризацию в backpropagation
   - Реализуйте Dropout в forward и backward pass
   - Попробуйте разные функции активации

4. **Эксперименты:**
   - Сравните скорость сходимости с разными learning rates
   - Исследуйте влияние глубины сети на обучение
   - Протестируйте на датасете MNIST

5. **Визуализация:**
   - Анимируйте процесс обучения (как меняется граница решений)
   - Визуализируйте веса сети
   - Постройте heatmap градиентов

6. **Продвинутое:**
   - Реализуйте автоматическое дифференцирование
   - Создайте вычислительный граф с динамическими операциями
   - Сравните производительность с PyTorch/TensorFlow